# Autograd experiments

In [1]:
import autograd.numpy as anp
import autograd as auto

In [2]:
def fun1(x0, x1):
    y = anp.exp(-2.0*x1 + x0)
    return y

In [3]:
fun1_x0 = auto.grad(fun1, 0)
fun1_x1 = auto.grad(fun1, 1)

In [4]:
print(fun1_x0(.5, .1))
print(fun1_x1(.5, .1))

1.34985880758
-2.69971761515


In [5]:
sigx = anp.array([[0, 1],[1,0]])
sigx

array([[0, 1],
       [1, 0]])

In [6]:
sigy = anp.array([[0, -1j],[1j,0]])
sigy

array([[ 0.+0.j, -0.-1.j],
       [ 0.+1.j,  0.+0.j]])

In [7]:
sigz = anp.array([[1, 0],[0, -1]])
sigz

array([[ 1,  0],
       [ 0, -1]])

In [8]:
sig_all = anp.vstack([sigx, sigy, sigz])
sig_all

array([[ 0.+0.j,  1.+0.j],
       [ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -0.-1.j],
       [ 0.+1.j,  0.+0.j],
       [ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j]])

In [9]:
sig_all = anp.reshape(sig_all, (3, 2, 2)).transpose(1, 2, 0)
sig_all

array([[[ 0.+0.j,  0.+0.j,  1.+0.j],
        [ 1.+0.j, -0.-1.j,  0.+0.j]],

       [[ 1.+0.j,  0.+1.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j]]])

In [10]:
ex = anp.array([1,0,0])
ey = anp.array([0,1,0])
ez = anp.array([0,0,1])
sigx_ = anp.dot(sig_all, ex)
sigy_ = anp.dot(sig_all, ey)
sigz_ = anp.dot(sig_all, ez)
print('sigx_=\n', sigx_)
print('sigy_=\n', sigy_)
print('sigz_=\n', sigz_)

sigx_=
 [[ 0.+0.j  1.+0.j]
 [ 1.+0.j  0.+0.j]]
sigy_=
 [[ 0.+0.j  0.-1.j]
 [ 0.+1.j  0.+0.j]]
sigz_=
 [[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


$ U = e^{i\sigma_3\theta_3} = C + i\sigma_3 S$

$S = \sin\theta_3, C = \cos \theta_3$

$\frac{dU}{dt} = \dot{\theta}_3(-S + i\sigma_3 C)$

abbreviations:

ax= axis

dwrt= derivative with respect to


In [11]:
def fun2(t, ax):
    assert ax in [1, 2, 3]
    return anp.eye(2)*anp.cos(t) + 1j*sig_all[:, :, ax-1]*anp.sin(t)
def fun2_t(t, ax):
    assert  ax in [1, 2, 3]
    return -anp.eye(2)*anp.sin(t) + 1j*sig_all[:, :, ax-1]*anp.cos(t)
def fun2r(t, ax):
    return anp.real(fun2(t, ax))
def fun2i(t, ax):
    return anp.imag(fun2(t, ax))
def jaco_fun2(t, ax):
    assert  ax in [1, 2, 3]
    return auto.jacobian(fun2r, 0)(t, ax) + 1j*auto.jacobian(fun2i, 0)(t, ax) 

In [12]:
for ax in range(1, 4):
    print(jaco_fun2(.5, 1) - fun2_t(.5, 1))

[[ 0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j]]
[[ 0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j]]
[[ 0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j]]


$ U = e^{i\sigma_k\theta_k} = C +
i\sigma_k \frac{\theta_k}{\theta} S$

$\theta = \sqrt{\theta_k\theta_k},
 S = \sin\theta, C = \cos \theta$

$\frac{dU}{dt}=-S \frac{\theta_k}{\theta}
 \dot{\theta_k}+ i\sigma_k\dot{\theta_r}
 \left[\frac{\theta_k\theta_r}{\theta^2} C+
  \frac{S}{\theta}(-\frac{\theta_k\theta_r}{\theta^2}
  + \delta_{k, r})\right]$

In [13]:
def fun3(t1, t2, t3):
    t = anp.sqrt(t1**2 + t2**2 + t3**2)
    tvec = anp.array([t1/t, t2/t, t3/t])
    return anp.eye(2)*anp.cos(t) + 1j*anp.dot(sig_all, tvec)*anp.sin(t) 

In [14]:
fun3(.1, .2, .3)

array([[ 0.93081287+0.29304884j,  0.19536589+0.09768295j],
       [-0.19536589+0.09768295j,  0.93081287-0.29304884j]])

In [15]:
fun3(0., 0., .3) - fun2(.3, ax=3)

array([[ 0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j]])

In [16]:
def fun3r(t1, t2, t3):
    return anp.real(fun3(t1, t2, t3))
def fun3i(t1, t2, t3):
    return anp.imag(fun3(t1, t2, t3))
# dwrt stands for "derivative with respect to". If dwrt=1, 2, 3 partial dwrt t1, t2, t3
def jaco_fun3(t1, t2, t3, dwrt):
    assert dwrt in [1, 2, 3]
    return auto.jacobian(fun3r, dwrt-1)(t1, t2, t3) + 1j*auto.jacobian(fun3i, dwrt-1)(t1, t2, t3) 

In [17]:
for k in range(1, 4):
    print(jaco_fun3(0., 0., .5, dwrt=k))

[[ 0.+0.j          0.+0.95885108j]
 [ 0.+0.95885108j  0.+0.j        ]]
[[ 0.00000000+0.j  0.95885108+0.j]
 [-0.95885108+0.j  0.00000000+0.j]]
[[-0.47942554+0.87758256j  0.00000000+0.j        ]
 [ 0.00000000+0.j         -0.47942554-0.87758256j]]


In [18]:
fun2_t(.5, ax=3)

array([[-0.47942554+0.87758256j,  0.00000000+0.j        ],
       [ 0.00000000+0.j        , -0.47942554-0.87758256j]])

In [19]:
def d_fun3(t1, t2, t3, dwrt):
    assert dwrt in [1, 2, 3]
    dwrt_ = dwrt - 1
    t = anp.sqrt(t1**2 + t2**2 + t3**2)
    tvec = anp.array([t1/t, t2/t, t3/t])
    dotted_vec = tvec*tvec[dwrt_]*anp.cos(t) + (anp.sin(t)/t)*(-tvec*tvec[dwrt_] + anp.eye(3)[dwrt_, :])
    return -anp.sin(t)*tvec[dwrt_]*anp.eye(2) + 1j*anp.dot(sig_all, dotted_vec)

In [20]:
for k in range(1, 4):
    print(jaco_fun3(1.1, .7, .5, dwrt=k) - d_fun3(1.1, .7, .5, dwrt=k))

[[ 0. -2.77555756e-17j  0. +5.55111512e-17j]
 [ 0. +5.55111512e-17j  0. +2.77555756e-17j]]
[[ 0. -1.38777878e-17j  0. +2.77555756e-17j]
 [ 0. +2.77555756e-17j  0. +1.38777878e-17j]]
[[ -5.55111512e-17+0.j  -1.38777878e-17+0.j]
 [  1.38777878e-17+0.j  -5.55111512e-17+0.j]]
